# Трансформеры
В этом домашнем задании мы рассмотим использование трансформеров в библиотеке PyTorch. Рассмотрим задачу языкового моделирования. Попробуем генерировать текст нейронной сетью. 

Ссылка на данные - https://drive.google.com/drive/folders/1x1A4ElliUGBPnHladGMwPxPuGxI8Vnpu?usp=sharing

In [8]:
# хороший тон, импортировать все необходимые библиотеки в одной ячейке ;)

import torch
from torch import nn

import numpy as np
import time

Что такое языковое моделирование? Это предсказание вероятности следующего токена (слова или буквы) на основе предыдущих токенов. Математически это можно описать так:

$$P(x_i|x_1, x_2 , ... , x_{i-1})$$ 

Последовательность $$ x_1, x_2, ... x_{i-1} $$ называют контекстом.

## Задание 0 (0 баллов, но сделать нужно)
Проставьте знаки неравенств, исходя из вашего опыта:
$$ P(раму | мама, мыла) > P(папу | мама, мыла) $$
"мама мыла раму" -- устойчивое выражение, а значит и более частотное

$$ P(столу | дорога, ложка, к) < P(обеду | дорога, ложка, к) $$
"дорога ложка к столу" -- устойчивое выражение, а значит и более частотное

$$ P(Евпатий | меня, зовут) < P(Ваня | меня, зовут) $$
евпатий менее часто встречающееся имя, чем ваня

$$ P(журналы | я, часто ,читаю) = P(комиксы | я, часто ,читаю) $$
здесь слова сопоставимы по частотности, что обусловливает сравнимую вероятность появления в таком контексте

Попробуйте объяснить выбор для каждого из примеров.

Ответ : выше

Если для некоторых из примеров проставить знаки достаточно просто, то на некоторые сложно сказать, какой овтет верный. Мы принимаем решение для данного задания исходя их опыта использования русского языка. Мы много читали на русском и слушали огромное количество русской речи. Обучение языковых моделей происходит по схожему принципу. 

Мы хотим показать модели столько текстов, сколько можем и надеемся, что она наберется достаточно опыта, чтобы расставлять такие знаки неравества максимально схоже с человеком.

## Задание 1 (0.5 балла)
Мы будем обучать языковую модель для предсказания следущей буквы. Такие языковые модели применяются в распозновании речи, так как предоставляют дополнительную информацию акустической модели при выборе следующего символа. Для начала, откройте файл с данными, посмотрите, какие символы входят в тексты, сколько их. Уберите из текста все символы переноса на новую строку и табуляцию.

In [9]:
path = 'small_corp_for_test.txt'
file = open(path, 'r')
data = file.readlines()
file.close()
len(data)

700000

In [10]:
print(*set(''.join(data)))
print('всего их', len(set(''.join(data))))
for index in range(len(data)):
    data[index] = data[index].replace('\n', '').replace('\t', '')

т х ч   п о й ф ц э р щ 
 д а у - е з ъ ь н я ы л ш с в к ю ё ж г и б м
всего их 36


## Задание 2 (0.5 балла)
Для обучения модели требуется сначала подготовить текст в подходящий для нейросети вид. Важно также отметить, что нужно добавить два токена start и end, которые отвечают за начало и конец текста. Используйте [ и ] для этой задачи. Также нам нужен токен pad, чтобы заполнять им текст до требуемой длинны для формирования батча.

Реализуйте метод preprocess класса Preprocessor. Он должен принимать на вход текст и длинну текста, которую мы ожидаем получить на выходе. Текст должен быть переведен в нижний регистр, в конец текста добавляется требуемое число pad токенов, далее текст векторизуется (каждому символу ставится свое число). Вернуть требуется два вектора. Полученный результат без последнего токена (на нем будем обучаться) и полученный результат без первого токена (целевые метки при обучении).

In [11]:
class Preprocessor:
    def __init__(self):
        self.alphabet = '_добсркгаупитнезчм яжлйвцыэь-шхюфщёъ][ '
        self.token2ind = {}
        self.ind2token = {}
        for i in range(len(self.alphabet)):
            self.token2ind[self.alphabet[i]] = i
            self.ind2token[i] = self.alphabet[i]
        
    
    def preprocess(self, text, window_size):
        text = text.lower()
        padded_text = text + '_' * (window_size + 1 - len(text))
        encoded_text = torch.tensor(list(map(lambda symbol: self.token2ind[symbol], padded_text)))
        return encoded_text[:-1], encoded_text[1:]

In [12]:
Preprocessor().preprocess('[мама]', 8)

(tensor([37, 17,  8, 17,  8, 36,  0,  0]),
 tensor([17,  8, 17,  8, 36,  0,  0,  0]))

## Задание 3 (0.5 балла)
Так как мы решили, что текст будет начинаться токеном [ и заканчиваться токеном ], данные нужно поправить. Реализуйте эту идею, добавьте данные токены в ваши тексты.

In [13]:
for index in range(len(data)):
    data[index] = '[' + data[index] + ']'

## Задание 4 (0.5 балла)
Так как мы не располагаем большими мощностями, то давайте ограничим максимальную длинну текста. Вы можете менять этот порог и тем самым уменьшать кол-во текстов в вашей выборке и увеличивая тем самым скорость обучения. Начнем же мы с 128. 
Выберите порог и оставьте только те тексты, длина которых не превосходит данный порог.

Далее разбейте тексты на train и test, перемешайте тексты при разбиении, размер тестовой выборки должен быть 15% от общего числа текстов. 

In [14]:
THRESHOLD = 10

short_data = list(filter(lambda x: len(x) <= THRESHOLD, data))
assert len(short_data) < len(data)

In [15]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(short_data, shuffle=True, test_size=0.15)

## Задание 5 (2 балла)
Напишем датасет. На вход датасету передается набор текстов, объект класса Preprocessor и размер окна, который вы выбрали в прошлом задании.
Реализуйте методы __len__ и __getitem__.

In [16]:
class TextDataset(torch.utils.data.Dataset):
    
    def __init__(self, x, preproc, win_size = 128):
        self.x = tuple(map(preproc.preprocess, x, [win_size] * len(x)))
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx]

In [17]:
preproc = Preprocessor()
train_dataset = TextDataset(train, preproc)
test_dataset = TextDataset(test, preproc)

## Задание 6 (2 балла)
Напишем модель. Класс для реализации positional encoding реализован за вас, он нужен, чтобы модель могла после получения эмбедингов понимать, на каком месте какой токен находится.

Заполните пропуски в классе модели. Гипперпараметры модели вам предлагается подобрать самостоятельно. Рекомендуется использовать не более 6 слоев в трансформере. В декореде испоьлзуйте две линейных слоя с функцией активации ReLU между ними.

## Задание 6_1 (0 баллов, но надо ответить!)
При обучении языковой модели на основе трансформеров мы используем маскирование символов (как мы это делаем - уже реализовано). Напишите, почему мы это делаем? Почему это так важно?

Благодаря механизму внимания модель при оценке вероятности следующего символа может обращаться к любому месту во входной последовательности. Необходимо запретить ей смотреть вперед: на те позиции, которые она еще не предсказала, иначе она будет просто смотреть в будущее и не будет учиться сама.

In [18]:
import math

class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [19]:
class LanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super(LanguageModel, self).__init__()
        self.emb = nn.Embedding(vocab_size, 100)
        self.pe = PositionalEncoding(100)
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(100, 10)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, 10)
        self.decoder = nn.Sequential(nn.Linear(100, 100), nn.ReLU(), nn.Linear(100, vocab_size))
    
    def forward(self, x, src_mask):
        x = self.pe(self.emb(x))
        x = x.transpose(1, 0)
        x = self.transformer_encoder(x, src_mask)
        x = self.decoder(x)
        return x.transpose(1, 0)
    
    def generate_square_subsequent_mask(self, sz):
        # А вот и то самое маскирование
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [20]:
model = LanguageModel(len('_добсркгаупитнезчм яжлйвцыэь-шхюфщёъ][ '))

## Задание 7 (2,5 балла)
Финишная прямая. Давайте реализуем класс для обучения модели и ее валидации. Следуйте указаниям в коде и заполните недостающие фрагменты в коде.

In [21]:
class Trainer:
    
    def __init__(self, model, train_dataset, test_dataset):
        
        self.model = model
        
        self.train_batch_size = 32
        self.test_batch_size = 32
        
        self.train_dataloader = torch.utils.data.DataLoader(train_dataset, 
                                                            batch_size=self.train_batch_size, 
                                                            shuffle=True)
        self.test_dataloader = torch.utils.data.DataLoader(test_dataset, 
                                                           batch_size=self.test_batch_size, 
                                                           shuffle=False)
        self.train_dataloader_size = len(train_dataset) // self.train_batch_size 
        self.test_dataloader_size = len(test_dataset) // self.test_batch_size
        
        self.device = 'cuda:0'
        self.criterion = nn.CrossEntropyLoss(ignore_index=0) # используйте CrossEntrophyLoss, передайте в качетсве параметра 
                             # ignore index индекс символа _, чтобы модель не штрафовалась за то
                             # что идет после закрывающего токена
        
        self.optimizer = torch.optim.Adam(model.parameters(), 1e-3)
        
        self.steps_to_print = 10
        
    def train_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        
        for batch in self.train_dataloader:
            x, y = batch
            model_output = self.model(x, self.model.generate_square_subsequent_mask(x.shape[1]))
            model_output = model_output.reshape(model_output.shape[0] * model_output.shape[1], model_output.shape[2])
            loss = self.criterion(model_output, y.reshape(y.shape[0] * y.shape[1]))
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()

            counted_loss += int(loss.unsqueeze(0).data)
            it += 1
            step += 1
            
            # реализуйте шаги обучения модели
            # сохраняйте значение ошибки в переменную counted_loss
            
            ################
            
            
            if step%self.steps_to_print == 0:
                result = 'Train epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.train_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
    
    def validate_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        for batch in self.test_dataloader:
            x, y = batch
            
            model_output = self.model(x, self.model.generate_square_subsequent_mask(x.shape[1]))
            model_output = model_output.reshape(model_output.shape[0] * model_output.shape[1], model_output.shape[2])
            loss = self.criterion(model_output, y.reshape(y.shape[0] * y.shape[1]))

            counted_loss += int(loss.unsqueeze(0).data)
            it += 1
            step += 1

            # реализуйте шаги для теста модели
            # помните, что данный метод уже запускается из 
            # блока with torch.no_grad(), а потому 
            # повторно его использовать не нужно
            
            ################
            
            if step%(self.steps_to_print//2) == 0:
                result = 'Validate epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.test_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
        
    def train(self, number_of_epochs):
        # model.to(self.device)
        for epoch in range(1, number_of_epochs+1):
            model.train()
            self.train_one_epoch(epoch)
            with torch.no_grad():
                model.eval()
                self.validate_one_epoch(epoch)
            print()

Что такое ppl? Перплексия. Ее можно интерпретировать как меру "удивленности" модели нужному символу. Чем меньше данная величина, тем лучше, ведь это значит, что модель если и сделала неправильный выбор, то не сильно удивлена своей ошибке.

Проведите несколько экспериментов, посмотрите, при каких гипперпараметрах значение перплексии минимально.

## Задание 8 (0.5 балла)
Запустите обучение на нескольких эпохах. Ориентируйтесь на ваши вычислительные мощности и время работы. Вы всегда можете посчитать, сколько секунд уходит на один батч.

In [ ]:
trainer = Trainer(model, train_dataset, test_dataset)
trainer.train(1)

Train epoch 1 | Step 10/1224 | Counted loss 26 | ppl 13.463738035001692 | time 49.654540061950684 | 
Train epoch 1 | Step 20/1224 | Counted loss 25 | ppl 12.182493960703473 | time 48.707382917404175 | 
Train epoch 1 | Step 30/1224 | Counted loss 24 | ppl 11.023176380641601 | time 47.98096036911011 | 
Train epoch 1 | Step 40/1224 | Counted loss 27 | ppl 14.879731724872837 | time 48.47192120552063 | 
Train epoch 1 | Step 50/1224 | Counted loss 27 | ppl 14.879731724872837 | time 48.17089819908142 | 
Train epoch 1 | Step 60/1224 | Counted loss 25 | ppl 12.182493960703473 | time 47.92381191253662 | 
Train epoch 1 | Step 70/1224 | Counted loss 25 | ppl 12.182493960703473 | time 48.08495855331421 | 
Train epoch 1 | Step 80/1224 | Counted loss 29 | ppl 18.17414536944306 | time 47.96441102027893 | 
Train epoch 1 | Step 90/1224 | Counted loss 24 | ppl 11.023176380641601 | time 48.00698661804199 | 
Train epoch 1 | Step 100/1224 | Counted loss 24 | ppl 11.023176380641601 | time 47.985023975372314 

KeyboardInterrupt: ignored

ну и так видно что лосс падает, перплексия вместе с ним, извините я не буду ждать все миллион лет....

## Задание 9 (1 балл)
Итак, давайте попробуем погенерировать текст нашей сеткой. Закончите функцию по генерации текста. Попробуйте сгенерировать какой-нибудь текст. Помните, что если вы хотите генерировать текст с нуля, то вы должны передать в качестве текста только токен start.
Прекратите генерировать текст, если модель выдала токен end или длинна текста больше 150.

In [41]:
def generate_text(text):
    x = []
    
    for letter in text:
        x.append(preproc.token2ind[letter])
    x = torch.from_numpy(np.array(x))
    
    pred = model(x.reshape(1, x.shape[0]), model.generate_square_subsequent_mask(x.shape[0]))
    ind = pred[0, -1, :].argmax()
    
    text += preproc.ind2token[int(ind)]
    
    if text[-1] == '_' or len(text) > 150:
        return text
    else:
        return generate_text(text)

In [42]:
generate_text('[ма')

'[маууууууууууиууууущуууууууууууууууууууууууууууууууууууууууууууууууууюууууууууууууууууууууюуууююууууууууюуууууууууууууууууюуууюуууууюуууууууюууууюууууу'

## Задание 10* (Задание - бонус, 5 баллов за реализацию при условии, что сделаны прошлые задания)
Давайте вспомним, что такое transfer learning. Мы хотим использовать уже предобученные эмбединги для нашей сети, чтобы наша сеть обучалась быстрее. Давайте попробуем обучить новую модель на уровне слов, а не символов, но для упрощения задачи используем предобученный слой из библиотеки Natasha, а вернее, ее блок Navec.

[Изучите](https://github.com/natasha/navec) то, как вставить слой в вашу нейронную сеть.

Теперь мы хотим, чтобы на вход модели подавались слова, модифицируйте ваш датасет. Возвращайте теперь номер слова в словаре navec.

In [3]:
path = 'small_corp_for_test.txt'
file = open(path, 'r')
data = file.readlines()
file.close()
print(len(data))

for index in range(len(data)):
    data[index] = data[index].replace('\n', '').replace('\t', '')

700000


In [4]:
! pip install navec -q

In [5]:
! wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

--2021-12-20 13:42:35--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar’

navec_hudlit_v1_12B 100%[===================>]  50.56M  16.2MB/s    in 3.1s    

2021-12-20 13:42:39 (16.2 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar’ saved [53012480/53012480]



In [6]:
from navec import Navec

NAVEC_PATH = '/content/navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(NAVEC_PATH)
navec.vocab['мамочка']

206063

In [7]:
class Preprocessor:
    def __init__(self, module):
        self.module = module
    
    def preprocess(self, text, window_size):
        text = text.lower().split()
        padded_text = text + ['паддинг'] * (window_size + 1 - len(text))
        encoded_text = torch.tensor(list(map(lambda word: self.module.vocab.get(word, 0), padded_text)))
        return encoded_text[:-1], encoded_text[1:]

In [8]:
preproc = Preprocessor(navec)
preproc.preprocess('Мама мыла раму', 10)

(tensor([205917, 223867, 366518,      0,      0,      0,      0,      0,      0,
              0]),
 tensor([223867, 366518,      0,      0,      0,      0,      0,      0,      0,
              0]))

In [9]:
class TextDataset_Navec(torch.utils.data.Dataset):
    
    def __init__(self, x, win_size = 128):
        # YOUR CODE HERE
        self.navec = Navec.load(NAVEC_PATH)
        self.x = list(map(lambda text: Preprocessor(self.navec).preprocess(text, win_size), x))
        # self.x = torch.tensor(self.x)
        ################
    
    def __len__(self):
        return len(self.x)
        ################
    
    def __getitem__(self, idx):
        return self.x[idx]
        ################

In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, shuffle=True, test_size=0.15)
train_dataset, test_dataset = TextDataset_Navec(train), TextDataset_Navec(test)

In [ ]:
train_dataset[0][0].shape

torch.Size([128])

Немного модифицируем модель. Теперь нам не нужны слои с трансформером, так как весь механизм внимания уже заложен в ембедингах. Давайте попробуем просто пройтись линейной головой над эмбедингами. Выберите параметры самостоятельно.

In [ ]:
import torch
from slovnet.model.emb import NavecEmbedding

emb = NavecEmbedding(navec)
input = torch.tensor([1, 2, 0])
output = emb(input)

In [14]:
! pip install slovnet

     |████████████████████████████████| 49 kB 2.5 MB/s 


In [18]:
from slovnet.model.emb import NavecEmbedding

class LanguageModel(nn.Module):
    def __init__(self):
        super(LanguageModel, self).__init__()
        self.emb_navec = NavecEmbedding(Navec.load(NAVEC_PATH))
        self.head = nn.Sequential(nn.Linear(300, 100), nn.ReLU(), nn.Linear(100, 1))
    
    def forward(self, x):
        x = self.emb_navec(x)
        x = self.head(x)
        return x

Теперь дело за малым! Надо немного модифицировать класс обучения, так как мы не используем маскирование, после чего можно приступить к тесту!

In [37]:
class Trainer:
    
    def __init__(self, model, train_dataset, test_dataset):
        
        self.model = model
        
        self.train_batch_size = 32
        self.test_batch_size = 32
        
        self.train_dataloader = torch.utils.data.DataLoader(train_dataset, 
                                                            batch_size=self.train_batch_size, 
                                                            shuffle=True)
        self.test_dataloader = torch.utils.data.DataLoader(test_dataset, 
                                                           batch_size=self.test_batch_size, 
                                                           shuffle=False)
        self.train_dataloader_size = len(train_dataset) // self.train_batch_size 
        self.test_dataloader_size = len(test_dataset) // self.test_batch_size
        
        self.device = 'cuda:0'
        self.criterion = nn.CrossEntropyLoss(ignore_index=0) # используйте CrossEntrophyLoss, передайте в качетсве параметра 
                             # ignore index индекс символа _, чтобы модель не штрафовалась за то
                             # что идет после закрывающего токена
        
        self.optimizer = torch.optim.Adam(model.parameters(), 1e-3)
        
        self.steps_to_print = 10
        
    def train_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0

        for batch in self.train_dataloader:
            x, y = batch
            
            model_output = self.model(x)
            model_output = model_output.reshape(model_output.shape[0] * model_output.shape[1], model_output.shape[2])
            loss = self.criterion(model_output, y.reshape(y.shape[0] * y.shape[1]))

            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()

            counted_loss += int(loss.unsqueeze(0).data)
            it += 1
            step += 1
            
            # реализуйте шаги обучения модели
            # сохраняйте значение ошибки в переменную counted_loss
            
            ################
            
            
            if step%self.steps_to_print == 0:
                result = 'Train epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.train_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
    
    def validate_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        for batch in self.test_dataloader:
            x, y = batch
            
            model_output = self.model(x)
            model_output = model_output.reshape(model_output.shape[0] * model_output.shape[1], model_output.shape[2])
            loss = self.criterion(model_output, y.reshape(y.shape[0] * y.shape[1]))

            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()

            counted_loss += int(loss.unsqueeze(0).data)
            it += 1
            step += 1
            
            # реализуйте шаги для теста модели
            # помните, что данный метод уже запускается из 
            # блока with torch.no_grad(), а потому 
            # повторно его использовать не нужно
            
            ################
            
            if step%(self.steps_to_print//2) == 0:
                result = 'Validate epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.test_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
        
    def train(self, number_of_epochs):
        model.to(self.device)
        for epoch in range(1, number_of_epochs+1):
            model.train()
            self.train_one_epoch(epoch)
            with torch.no_grad():
                model.eval()
                self.validate_one_epoch(epoch)
            print()

Запустите обучение. 

In [20]:
model = LanguageModel()
trainer = Trainer(model, train_dataset, test_dataset)
trainer.train(1)

Train epoch 1 | Step 10/273 | Counted loss 30 | ppl 20.085536923187668 | time 94.80115747451782 | 
Train epoch 1 | Step 20/273 | Counted loss 29 | ppl 18.17414536944306 | time 93.62750244140625 | 


KeyboardInterrupt: ignored

запустила, но с вашего позволения не буду ждать его завершения